In [66]:
import pandas as pd
import xlsxwriter
import glob
import re

In [67]:
files = [str(file).split('\\')[-1] for file in glob.glob('./CSVs/*.csv')]

companies = set( str(file).split('_')[0] for file in files )

formated_company_names = [name.replace('-', ' ').title().strip() for name in companies]

col_names = ['a', 'Title', 'Acceptance', 'Level', 'e', 'Link']

In [68]:
def excel_hyperlink(title, link):
    return '=HYPERLINK("{}", "{}")'.format(link, title)

def make_excel(company):
    df = pd.concat(map(lambda x: pd.read_csv(x, names=col_names), 
                       glob.glob('CSVs/' + company + '_*.csv')), ignore_index=True)

    
    df['Title'] = df['Title'].apply(lambda x: str(x).strip())
    df['Link'] = df['Link'].apply(lambda x: str(x).strip())
    
    df = df.drop_duplicates(keep="first", subset=['Title', 'Link'])
    df['Title'] = df.apply(lambda x: excel_hyperlink(x.Title, x.Link), axis=1)
    
    df = df.drop(['a', 'e', 'Link'], axis=1)
    
    df = df.sort_values(by=['Acceptance'], ascending=False)
    df['Level'] = pd.Categorical(df['Level'],categories=['Easy', 'Medium', 'Hard'],ordered=True)
    df = df.sort_values('Level', kind='stable', ascending=True)
    
    df.to_excel('./Excels/' + company + '.xlsx', header=True, index=False, index_label=False, engine='xlsxwriter')


In [69]:
def make_md(company):
    df = pd.concat(map(lambda x: pd.read_csv(x, names=col_names), 
                       glob.glob('CSVs/' + company + '_*.csv')), ignore_index=True)

    
    df['Title'] = df['Title'].apply(lambda x: str(x).strip())
    df['Link'] = df['Link'].apply(lambda x: str(x).strip())
    
    df = df.drop_duplicates(keep="first", subset=['Title', 'Link'])
    df['Title'] = df.apply(lambda x: '['+x.Title+']('+x.Link+')', axis=1)
    
    df = df.drop(['a', 'e', 'Link'], axis=1)
    
    df = df.sort_values(by=['Acceptance'], ascending=False)
    df['Level'] = pd.Categorical(df['Level'],categories=['Easy', 'Medium', 'Hard'],ordered=True)
    df = df.sort_values('Level', kind='stable', ascending=True)
    
    df.to_markdown(buf='./list/' + company + '.md', index=False, tablefmt="github")

for company in companies:
    make_md(company)
#    make_excel(company)